In [ ]:
import AI_EPI
# step1 prepare
basedir = "/home/wyh/scdata/combined_data/Epithelial"
output_directory = basedir+"/NMFV2/"
run_name = "Epithelial_adeno"
countfn = "/home/wyh/scdata/combined_data/Epithelial/NMFV2/adata_adeno_p_sample.h5ad"
K = ' '.join([str(i) for i in range(20,50)])
seed = 2022
numiter = 10
numworkers = 100
# select important genes
gene_HRG = HighlyRegionalGenes(sc.read_h5ad("adata_adeno_p_sample.h5ad"),2000)
gene_HRG.to_csv("./NMFV2/adeno_P_HRG.csv",index = False,header = False)
genes_file = basedir+"/NMFV2/adeno_P_HRG.csv"

prepare_cmd = '/home/wyh/anaconda3/envs/wuyh_py38/bin/python cnmf.py prepare --output-dir %s --name %s -c %s -k %s --n-iter %d --total-workers %d --seed %d --genes-file %s --beta-loss frobenius' % (output_directory, run_name, countfn, K, numiter, numworkers, seed, genes_file)
print('Prepare command assuming parallelization with %d cores:\n%s' % (numworkers, prepare_cmd))
! {prepare_cmd}

In [ ]:
"""## Submitting all of the factorization jobs for the 0th (I.e. the only) worker
factorize_cmd = '/home/unix/kjag/.conda/envs/pegasus/bin/python cnmf.py factorize --output-dir %s --name %s --worker-index 0' % (output_directory, run_name)
print('Factorize command with no parallelization:\n%s' % factorize_cmd)
!{factorize_cmd}
"""
## Using GNU parallel
## This took 4 minutes in our testing
basedir = "/home/wyh/scdata/combined_data/Epithelial"
output_directory = basedir+"/NMFV2/"
worker_index = ' '.join([str(x) for x in range(numworkers)])
factorize_cmd = '/home/wyh/anaconda3/envs/wuyh_py38/bin/python %s/cnmf.py factorize --output-dir %s --name %s --worker-index %s --total-workers %d'
commands = []
for i in worker_index.split(' '):
    commands.append(factorize_cmd%(basedir, output_directory, run_name, i, numworkers))
commands = "\n".join(commands)
print('Factorize command to simultaneously run factorization over %d cores using GNU parallel:\n%s' % (numworkers, commands))

joblist = open("/home/wyh/scdata/combined_data/Epithelial/NMFV2/joblist", "w")
joblist.write(commands)
joblist.close()
!{commands}

In [ ]:
# step2 combine
cmd = '/home/wyh/anaconda3/envs/wuyh_py38/bin/python /home/wyh/scdata/combined_data/Epithelial/cnmf.py combine --output-dir %s --name %s' % (output_directory, run_name)
print(cmd)
!{cmd}

In [ ]:
# step3 k selection
worker_index = ' '.join([str(x) for x in range(1)])
kselect_plot_cmd = '/home/wyh/anaconda3/envs/wuyh_py38/bin/python /home/wyh/scdata/combined_data/Epithelial/cnmf.py k_selection_plot --output-dir %s --name %s' % (output_directory, run_name)
print('K selection plot command: %s' % kselect_plot_cmd)
!{kselect_plot_cmd}

from IPython.display import Image
Image(filename = "/home/wyh/scdata/combined_data/Epithelial/NMFV2/%s/%s.k_selection.png"%(run_name, run_name), width=1000, height=1000)

In [ ]:
# step4 consenseus
for K in [36,44,46]:
    selected_K = K
    density_threshold = 2
    consensus_cmd = '/home/wyh/anaconda3/envs/wuyh_py38/bin/python /home/wyh/scdata/combined_data/Epithelial/cnmf.py consensus --output-dir %s --name %s --local-density-threshold %.2f --components %d --show-clustering' % (output_directory, run_name, density_threshold, selected_K)
    print('Consensus command for K=%d:\n%s' % (selected_K, consensus_cmd))
    !{consensus_cmd}

    density_threshold_str = ('%.2f' % density_threshold).replace('.', '_')
    Image(filename = "./NMFV2/%s/%s.clustering.k_%d.dt_%s.png" % (run_name, run_name, selected_K, density_threshold_str), width=1000, height=1000)

In [ ]:
# step5 result show
selected_K = 26
density_threshold = 2.00
density_threshold_str = ('%.2f' % density_threshold).replace('.', '_')
run_name = "Epithelial_adeno"
usage = pd.read_csv('./NMFV2/%s/%s.usages.k_%d.dt_%s.consensus.txt' % (run_name, run_name, selected_K, density_threshold_str),
                    sep='\t', index_col=0)
usage.columns = ['Usage_%s' % i for i in usage.columns]
usage_norm = usage.div(usage.sum(axis=1), axis=0)

adata_adeno_sample = sc.read_h5ad("./NMFV2/adata_adeno_p_sample.h5ad")
adata_adeno_sample.obs = pd.merge(left=adata_adeno_sample.obs, right=usage_norm, how='left', left_index=True, right_index=True)
sc.pl.umap(adata_adeno_sample, color=list(usage_norm.columns) , ncols=3, vmin=0, vmax=1, save="tumor_%d.png"%selected_K)

In [ ]:
selected_K = 23
density_threshold = 2.00
run_name = "Epithelial_adeno"
density_threshold_str = ('%.2f' % density_threshold).replace('.', '_')
gene_scores = pd.read_csv('./NMFV2/%s/%s.gene_spectra_score.k_%d.dt_%s.txt' % (run_name, run_name, selected_K, density_threshold_str),
                sep='\t', index_col=0).T
gene_scores.dropna(inplace =True)
top_genes = []
ngenes = 200
for gep in gene_scores.columns:
    top_genes.append(list(gene_scores.sort_values(by=gep, ascending=False).index[:ngenes]))

top_genes = pd.DataFrame(top_genes, index=gene_scores.columns).T
top_genes

In [ ]:
## score each cell by gene module
adata_adeno = sc.read_h5ad("./adata_adeno_P.h5ad")
ngenes = 100

K_range = [23]
for K in K_range:
    program_num = K

    top_genes = pd.read_csv("./NMFV2/top_genes_GM"+str(program_num)+".csv",index_col=0)
    top_genes = top_genes.iloc[range(ngenes),]
    top_genes.shape

    for iGM in range(top_genes.shape[1]):
        GM =list(top_genes.iloc[:,iGM])
        sc.tl.score_genes(adata_adeno,gene_list = GM,score_name = "score_GM"+str(iGM+1))
    score_module = adata_adeno.obs[["score_GM"+str(i+1) for i in range(top_genes.shape[1])]]
    score_module.to_csv("./gene_module/adeno_p/patient_GM_score_" + str(program_num) +"GM_"+str(ngenes)+"genes.csv")